In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.setrecursionlimit(10000)
from IPython.display import clear_output
sys.path.append("..")

from GameFlow import Board, BoardTerminalTest, FenceChecker, MoveChecker
from Search import MinimaxSearch, AlphaBetaSearch, DLMinimaxSearch, DLAlphaBetaSearch
from Heuristics import DistanceHeuristic
from UIKit import UIBoard
from Core import Player

grid_size = 5
fence_length = 2

fence_checker = FenceChecker(
    grid_size = grid_size,
    fence_length = fence_length
)
move_checker = MoveChecker(grid_size)

board = Board(
    grid_size = grid_size,
    fence_checker = fence_checker,
    player_positions = None,
    move_checker = move_checker,
    fences_horizontal=set(),
    fences_vertical=set()
)
terminal_test = BoardTerminalTest()
heuristic = DistanceHeuristic()
depth = 1

In [2]:
search = DLAlphaBetaSearch(
    depth = depth,
    heuristic = heuristic
)

In [3]:
fences_horizontal = set({ (3, 2)})
fences_vertical = set()
player_positions = {Player.MAX: (2, 0), Player.MIN: (2, 4)}

board = Board(
    grid_size = grid_size,
    fence_checker = fence_checker,
    player_positions = player_positions,
    move_checker = move_checker,
    fences_horizontal=fences_horizontal,
    fences_vertical=fences_vertical
)

ui_board = UIBoard(
    board_size = grid_size,
    fences_horizontal=fences_horizontal,
    fences_vertical=fences_vertical,
    player_positions = player_positions
)

In [4]:
ui_board.print()

    0   1   2   3   4
  ---------------------
0 |   |   | X |   |   | 
  --------------------
1 |   |   |   |   |   | 
  --------------------
2 |   |   |   |   |   | 
  -------------===-===
3 |   |   |   |   |   | 
  --------------------
4 |   |   | O |   |   | 
  ---------------------


In [5]:
heuristic(board)

4

In [6]:
from Protocols import Search, State, TerminalTest

In [129]:
import heapq
from abc import ABC, abstractmethod
from random import shuffle

class Node:
    def __init__(self,
                 parent = None,
                 action = None,
                 state = None,
                 value = 0,
                 depth = 0,
                 cost_to_root = 0):
        self.parent = parent
        self.action = action
        self.state = state
        self.depth = depth
        
        # this property represents the node's value
        self.value = value 
        
        # this property represents the cost from the node to the root
        # in lectures, this is denoted as g(n)
        self.cost_to_root = cost_to_root 

    # Need to define this function in order for the queue to work, so that the heapq knows 
    # how to compare values of type Node
    def __lt__(self, other):
        if self.value == None or other.value == None:
            return False
        else:
            return self.value < other.value
        
    def __le__(self, other):
        return self.value <= other.value
    
    def __ge__(self, other):
        return self.value >= other.value
        
    def __repr__(self):
        return f"Node({self.value})"

class Frontier(ABC):
    
    def __init__(self):
        self.frontier = deque()
        self.total_nodes = 0
    
    @abstractmethod
    def add(self, node):
        self.total_nodes += 1
        pass
    
    def clear(self):
        if type(self.frontier) == "deque":
            self.frontier = deque()
        else:
            self.frontier = []
        
    def isEmpty(self) -> bool:
        return len(self.frontier) == 0

    def pop(self):
        if not self.isEmpty():
            return self.frontier.popleft()
        else: return
        
    def __len__(self):
        return len(self.frontier)
    
    def __repr__(self):
        return f"{self.frontier}"
    
class NodeFunction: 
    # This method evaluates the function 
    def evaluate(self, node: Node) -> int:
        pass

class AStarFunction(NodeFunction): 
    
    # We need to specify the initializer, as we need to pass in here the 
    # heuristsic function
    def __init__(self, heuristicFunction):
        self.heuristicFunction = heuristicFunction
    
    def evaluate(self, node: Node) -> int:
        # g(n) + h(n)
        return node.cost_to_root + self.heuristicFunction(node.state)    


class BestFirstFrontier(Frontier):
    
    def __init__(self, nodeFunction: NodeFunction):
        self.frontier = []
        self.total_nodes = 0
        self.nodeFunction = nodeFunction
    
    def add(self, node):
        super().add(node)
        node.value = self.nodeFunction.evaluate(node)
        heapq.heappush(self.frontier, node)
    
    def pop(self):
        if not self.isEmpty():
            return heapq.heappop(self.frontier)
        else:
            return None

class GraphSearch(Search):
    
    def __init__(self, frontier):
        self.reached = set()
        self.frontier = frontier
        super().__init__()
    
    def find_strategy(self, initial_state: State, goal_test: TerminalTest):
        node = Node(parent = None, 
                    action = None, 
                    state = initial_state, 
                    value = 0,
                    depth = 0,
                    cost_to_root = 0)
        
        node.value = self.frontier.nodeFunction.evaluate(node)
        self.frontier.add(node)
        while not self.frontier.isEmpty():
            node = self.frontier.pop()
            if node.state not in self.reached:
                if goal_test.is_terminal(node.state):
                    return node
                
                self.reached.add(node.state)
                
                applicable_actions = list(node.state.get_applicable_actions())
                for action in applicable_actions:
                    new_state = node.state.get_action_result(action)
                        
                    child = Node(parent = node,
                                 action = action,
                                 state = new_state,
                                 cost_to_root = node.cost_to_root)
                    
                    if new_state not in self.reached:
                        self.frontier.add(child)
        return None   


In [130]:
heuristic = DistanceHeuristic()
node_function = AStarFunction(heuristic)
frontier = BestFirstFrontier(node_function)
search = GraphSearch(frontier)

In [131]:
board = Board(
    grid_size = grid_size,
    fence_checker = fence_checker,
    player_positions = None,
    move_checker = move_checker,
    fences_horizontal={(2,1)},
    fences_vertical={(1,0)},
    disable_turn = True
)
UIBoard.pring_board(board)

    0   1   2   3   4
  ---------------------
0 |   |   ‖ X |   |   | 
  --------------------
1 |   |   ‖   |   |   | 
  ---------===-===----
2 |   |   |   |   |   | 
  --------------------
3 |   |   |   |   |   | 
  --------------------
4 |   |   | O |   |   | 
  ---------------------


In [132]:
node = search.find_strategy(board, terminal_test)

In [133]:
tmp_node = node
actions = []
values = []
while True:
    actions.append(tmp_node.action)
    values.append(tmp_node.value)
    if tmp_node.parent:
        tmp_node = tmp_node.parent
    else:
        break
actions = actions[::-1]
values = values[::-1]

In [134]:
len(actions) - 1

6

In [118]:
UIBoard.pring_board(board)

    0   1   2   3   4
  ---------------------
0 |   |   ‖ X |   |   | 
  --------------------
1 |   |   ‖   |   |   | 
  ---------===-===----
2 |   |   |   |   |   | 
  --------------------
3 |   |   |   |   |   | 
  --------------------
4 |   |   | O |   |   | 
  ---------------------


In [39]:
ui_board = UIBoard(
    board_size = grid_size,
    fences_horizontal=fences_horizontal,
    fences_vertical=fences_vertical,
    player_positions = player_positions
)
ui_board.print()

    0   1   2   3   4
  ---------------------
0 |   |   | X |   |   | 
  --------------------
1 |   |   |   |   |   | 
  --------------------
2 |   |   |   |   |   | 
  -------------===-===
3 |   |   |   |   |   | 
  --------------------
4 |   |   | O |   |   | 
  ---------------------
